In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

In [2]:
## demo optimization problem

# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 1.0
optimal var 1.0 1.570086213240983e-22


In [3]:
# set up parameters

A_p = np.array([[1,0,-1],[-1,1,0],[0,-1,1]]) #edge-node incidence matrix for power
A_g = A_p #same for gas

In [22]:
# parameters

generators = pd.DataFrame()
generators["name"] = ["coal","gas","pv"]
generators["node_p"] = [0,1,2]
generators["node_g"] = [0,1,2]
generators["is_gas"] = [False,True,False]
generators["min_cap_mw"] = [0,0,0]
generators["max_cap_mw"] = [100,200,50]
generators["fuel_cost"] = [5,10,50]
generators["efficiency"] = [5,5,5]

lines = pd.DataFrame()
lines["from_node"] = [0,1,2]
lines["to_node"] = [1,2,0]
lines["reactance_pu"] = [0.3,0.3,0.3]
lines["capacity_mw"] = [500,500,500]

load = pd.DataFrame()
load["node"] = [0,1,2]
load["load_mw"] = [0,0,40]

p_hat_mw = 100 #MW
voltage_angle_cap_radians = 2 * np.pi
slack_node_index = 0
nse_cost = 1000 # penalty for non served energy

gas_supply = pd.DataFrame()
gas_supply["node_g"] = [0,1,2]
gas_supply["supply_ej"] = [1000,1000,1000] #exajoules

gas_demand = pd.DataFrame()
gas_demand["node_g"] = [0,1,2]
gas_demand["demand_ej"] = [100,100,100]

pipelines = pd.DataFrame()
pipelines["from_node"] = [0,1,2]
pipelines["to_node"] = [1,2,0]
pipelines["capacity_ej"] = [100,100,100]
pipelines["cost"] = [5,5,5]

# sets
nodes_p = list(range(A_p.shape[0]))
edges_p = list(range(A_p.shape[0]))
nodes_g = list(range(A_g.shape[0]))
edges_g = list(range(A_g.shape[0]))
gens = list(generators.index)

In [25]:
generators.to_csv("generators.csv",index=False)
lines.to_csv("lines.csv",index=False)
load.to_csv("load.csv",index=False)
gas_supply.to_csv("gas_supply.csv",index=False)
gas_demand.to_csv("gas_demand.csv",index=False)
pipelines.to_csv("pipelines.csv",index=False)

In [23]:
# decision variables

# TODO: add cost based on absolute value of flow

# voltage angle at each node (radians)
v_n = cp.Variable(len(nodes_p))

# generation
gen = cp.Variable(len(generators),nonneg=True)

# slack generation for power
slack_p = cp.Variable(len(nodes_p),nonneg=True)

# gas supply
supply_gas = cp.Variable(len(nodes_g),nonneg=True)

# gas flow
flow_gas = cp.Variable(len(edges_g))

# gas flow positive component
flow_gas_pos = cp.Variable(len(edges_g),nonneg=True)

# gas flow negative component
flow_gas_neg = cp.Variable(len(edges_g),nonneg=True)

# slack demand for gas
slack_g = cp.Variable(len(nodes_g),nonneg=True)

# expressions

# power flow on an edge
p_e = []

for e in edges_p:
    p_e += [p_hat_mw * (1/lines.reactance_pu[e]) * sum(A_p[n,e]*v_n[n] for n in nodes_p)]

# constraints

constraints = []

# power energy balance
for n in nodes_p:
    constraints += [
        sum(gen[g] if generators.loc[g].node_p == n else 0 for g in gens)
        - sum(A_p[n,e]*p_e[e] for e in edges_p)
        - load.loc[n].load_mw
        - slack_p[n]
        == 0
    ]

# tx line limits
for e in edges_p:
    constraints += [
        -1 * lines.loc[e].capacity_mw <= p_e[e]
    ]
    constraints += [
        p_e[e] <= lines.loc[e].capacity_mw
    ]

# voltage angle differential limits
for e in edges_p:
    constraints += [
        -1 * voltage_angle_cap_radians <= sum(A_p[n,e]*v_n[n] for n in nodes_p)
    ]
    constraints += [
        sum(A_p[n,e]*v_n[n] for n in nodes_p) <= voltage_angle_cap_radians
    ]

# slack bus
constraints += [v_n[slack_node_index] == 0]

# generator limits
for g in gens:
    constraints += [
        generators.loc[g].min_cap_mw <= gen[g]
    ]
    constraints += [
        gen[g] <= generators.loc[g].max_cap_mw
    ]
    
# gas energy balance
for n in nodes_g:
    constraints += [
        supply_gas[n]
        - sum(A_g[n,e]*flow_gas[e] for e in edges_g)
        - gas_demand.loc[n].demand_ej
        - sum(gen[g]*generators.loc[g].efficiency if generators.loc[g].is_gas else 0 for g in gens )
        - slack_g[n]
        == 0
    ]
    
# gas supply limits
for n in nodes_g:
    constraints += [
        0 <= supply_gas[n]
    ]
    constraints += [
        supply_gas[n] <= gas_supply.loc[n].supply_ej
    ]
    
# gas flow constraints
for e in edges_g:
    constraints += [
        -1 * pipelines.loc[e].capacity_ej <= flow_gas[e]
    ]
    constraints += [
        flow_gas[e] <= pipelines.loc[e].capacity_ej
    ]
    
# set up absolute value variables
for e in edges_g:
    constraints += [
        flow_gas_pos[e] >= flow_gas[e]
    ]
    constraints += [
        flow_gas_neg[e] >= -1 * flow_gas[e]
    ]

# objective
obj = cp.Minimize(
    sum(gen[g]*generators.loc[g].fuel_cost for g in gens)
    + sum(slack_p[n]*nse_cost for n in nodes_p)
    + sum((flow_gas_pos[e] + flow_gas_neg[e]) * pipelines.loc[e].cost for e in edges_g) 
    + sum(slack_g[n]*nse_cost for n in nodes_g)
)


# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", v_n[0].value, v_n[1].value, v_n[2].value)

status: optimal
optimal value 200.0000000840339
optimal var -9.757911193358136e-21 -0.03999999999540575 -0.0799999999974259
